In [1]:

import os, sys, glob, subprocess, pathlib
import numpy as np
import pandas as pd
import xarray as xr
from mo_evaluation import mo_evaluate
from MOASMO_parameters import get_parameter_from_Namelist_or_lndin




def read_parameter_csv(file_parameter_list):
    df_calibparam = pd.read_csv(file_parameter_list)

    for c in ['Upper', 'Lower', 'Factor', 'Value']:
        if c in df_calibparam.columns:
            if isinstance(df_calibparam.iloc[0][c], str):
                arr = []
                for i in range(len(df_calibparam)):
                    vi = df_calibparam.iloc[i][c]
                    if ',' in vi:
                        arr.append(np.array(vi.split(',')).astype(np.float64))
                    elif '[' in vi:
                        arr.append(np.array(vi.strip('[]').replace('\n', '').split(), dtype=np.float64))
                    else:
                        try:
                            arr.append(np.array([np.float64(vi)]))
                        except:
                            arr.append(np.array([-99999]))
                df_calibparam[c] = arr

    return df_calibparam

def xmlquery_output(pathCTSM, keyword):
    os.chdir(pathCTSM)
    out = subprocess.run(f'./xmlquery {keyword}', shell=True, capture_output=True)
    out = out.stdout.decode().strip().split(' ')[-1]
    return out


def clone_CTSM_model_case(script_clone, source_modelcase, target_modelcase, target_cimeoutput=''):
    if len(target_cimeoutput) > 0:
        settings = f'--cime-output-root {target_cimeoutput}'
    else:
        settings = ''
    _ = subprocess.run(f"{script_clone} --case {target_modelcase} --clone {source_modelcase} {settings}", shell=True)
    os.chdir(target_modelcase)
    _ = subprocess.run('./xmlchange BUILD_COMPLETE=TRUE', shell=True)


def update_CTSM_parameter_ParamFile(param_names, param_values, path_CTSM_case, outfile_newparam):
    # param_names and param_values: list
    file_param_base = ''
    # file_user_nl_clm = f'{path_CTSM_clone}/user_nl_clm'
    # with open(file_user_nl_clm) as f:
    #     for line in f:
    #         line = line.strip()
    #         if line.startswith('paramfile'):
    #             file_param_base = line.split('=')[-1].strip().replace('\'', '')
    if len(file_param_base) == 0:
        infile_lndin = f'{path_CTSM_case}/Buildconf/clmconf/lnd_in'
        with open(infile_lndin, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('paramfile'):
                    file_param_base = line.split('=')[-1].strip().replace('\'', '')

    ds_param = xr.load_dataset(file_param_base)
    for i in range(len(param_names)):
        pn = param_names[i]
        vnew = param_values[i]
        if not pn in ds_param.data_vars:
            print(f'Error!!! Variable {pn} is not find in parameter file {file_param_base}!!!')
            sys.exit()
        else:
            vold = ds_param[pn].values
            print(f'  -- Updating parameter {pn}: old mean value is {np.nanmean(vold)}. New mean value {np.nanmean(vnew)}')
            if vnew.size==1 and ds_param[pn].values.size>1:
                print(f'Warning! New parameter size=1, but raw parameter size={ds_param[pn].values.size}. Force the mean equal to the new parameter value.')
                m = np.nanmean(ds_param[pn].values)
                ds_param[pn].values = ds_param[pn].values - m + np.squeeze(vnew)
            else:
                ds_param[pn].values = np.squeeze(vnew)
    # write to new parameter file
    ds_param.to_netcdf(outfile_newparam, format='NETCDF3_CLASSIC')

def update_CTSM_parameter_NamelistFile(param_names, param_values, file_namelist):
    if len(param_names) > 0:
        with open(file_namelist, 'a') as f:
            print('# Writing new parameter values')
            for i in range(len(param_names)):
                    _ = f.write(f'{param_names[i]} = {np.squeeze(param_values[i])}\n')
                    print(f'  -- Writing parameter {param_names[i]} to namelist file. New value is {param_values[i]}')


def update_CTSM_parameter_SurfdataFile(param_names, param_values, path_CTSM_case, outfile_newsurfdata):

    file_surfdata = ''
    # file_user_nl_clm = f'{path_CTSM_clone}/user_nl_clm'
    # with open(file_user_nl_clm) as f:
    #     for line in f:
    #         line = line.strip()
    #         if line.startswith('fsurdat'):
    #             file_surfdata = line.split('=')[-1].strip().replace('\'', '')
    if len(file_surfdata) == 0:
        infile_lndin = f'{path_CTSM_case}/Buildconf/clmconf/lnd_in'
        with open(infile_lndin, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('fsurdat'):
                    file_surfdata = line.split('=')[-1].strip().replace('\'', '')

    ds_surf = xr.load_dataset(file_surfdata)
    for i in range(len(param_names)):
        pn = param_names[i]
        vnew = param_values[i]
        if not pn in ds_surf.data_vars:
            print(f'Error!!! Variable {pn} is not find in parameter file {file_surfdata}!!!')
            sys.exit()
        else:
            vold = ds_surf[pn].values
            print(f'  -- Updating surfdata {pn}: old mean value is {np.nanmean(vold)}. New mean value {np.nanmean(vnew)}')
            if vnew.size==1 and ds_surf[pn].values.size>1:
                print(f'Warning! New surfdata size=1, but raw surfdata size={ds_surf[pn].values.size}. Force the mean equal to the new surfdata value.')
                m = np.nanmean(ds_surf[pn].values)
                ds_surf[pn].values = ds_surf[pn].values - m + np.squeeze(vnew)
            else:
                ds_surf[pn].values = np.squeeze(vnew)

    ds_surf.to_netcdf(outfile_newsurfdata, format='NETCDF3_CLASSIC')


def update_ctsm_parameters(path_CTSM_case, file_parameter_set):

    file_user_nl_clm = f'{path_CTSM_case}/user_nl_clm'
    # df_calibparam = pd.read_csv(file_parameter_set)
    df_calibparam = read_parameter_csv(file_parameter_set)

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Namelist']
    if len(dfi) > 0:
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_NamelistFile(param_names, param_values, file_user_nl_clm)

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Param']
    if len(dfi) > 0:
        outfile_newparam = f'{path_CTSM_case}/updated_parameter.nc'
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_ParamFile(param_names, param_values, path_CTSM_case, outfile_newparam)
        # add to the name list file
        with open(file_user_nl_clm, 'a') as f:
            f.write(f"paramfile='{outfile_newparam}'\n")

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Surfdata']
    if len(dfi) > 0:
        outfile_newsurfdata = f'{path_CTSM_case}/updated_surfdata.nc'
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_SurfdataFile(param_names, param_values, path_CTSM_case, outfile_newsurfdata)
        # add to the name list file
        with open(file_user_nl_clm, 'a') as f:
            f.write(f"fsurdat='{outfile_newsurfdata}'\n")

    print('Successfully update parameters!')

In [2]:

script_clone = '/glade/u/home/guoqiang/CTSM_repos/CTSM/cime/scripts/create_clone'
path_CTSM_base = '/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100'
file_parameter_set = '/glade/scratch/guoqiang/moasmo_test/param_sets/paramset_iter0_trial0.csv'
path_archive = '/glade/scratch/guoqiang/moasmo_test/ctsm_outputs'
caseflag = 'iter0_trial0'

delete_clone = True

In [3]:

# clone case
path_CTSM_base = str(pathlib.Path(path_CTSM_base))
path_CTSM_clone = f'{path_CTSM_base}_{caseflag}'
clone_case_name = pathlib.Path(path_CTSM_clone).name
clone_CTSM_model_case(script_clone, path_CTSM_base, path_CTSM_clone)

 Successfully created new case CAMELS_100_iter0_trial0 from clone case CAMELS_100 
Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
job is case.run USER_REQUESTED_WALLTIME None USER_REQUESTED_QUEUE None WALLTIME_FORMAT %H:%M:%S
Creating batch scripts
Writing case.run script from input template /glade/u/home/guoqiang/CTSM_repos/CTSM/ccs_config/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/u/home/guoqiang/CTSM_repos/CTSM/ccs_config/machines/template.st_archive
Creating file case.st_archive
If an old case build already exists, might want to run 'case.build --clean' before building
You can now run './preview_run' to get more info on how your case will be run
For your changes to take effect, run:
./case.build --clean-all
./case.build


In [4]:
# change parameters, which won't affect files of path_CTSM_base
update_ctsm_parameters(path_CTSM_clone, file_parameter_set)

# Writing new parameter values
  -- Writing parameter baseflow_scalar to namelist file. New value is [0.09626875]
  -- Writing parameter upplim_destruct_metamorph to namelist file. New value is [224.375]
  -- Writing parameter precip_repartition_nonglc_all_rain_t to namelist file. New value is [3.39]
  -- Writing parameter precip_repartition_glc_all_rain_t to namelist file. New value is [1.39]
  -- Writing parameter precip_repartition_glc_all_snow_t to namelist file. New value is [-0.61]
  -- Writing parameter precip_repartition_nonglc_all_snow_t to namelist file. New value is [1.39]
  -- Updating parameter vcmaxha: old mean value is 23745.542400000002. New mean value 20575.0
  -- Updating parameter om_frac_sf: old mean value is 0.8319456. New mean value 0.446875
  -- Updating parameter slopebeta: old mean value is -2.8477065. New mean value -3.99125
  -- Updating parameter fff: old mean value is 3.1866575. New mean value 5.329675
  -- Updating parameter e_ice: old mean value is 6.7005

In [15]:
# run model in "no-batch" mode
os.chdir(path_CTSM_clone)
subprocess.run('./case.submit --no-batch', shell=True)

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
  2023-06-01 22:11:12 atm 
Create namelist for component datm
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/../components/cdeps/datm/cime_config/buildnml
  2023-06-01 22:11:12 lnd 
Create namelist for component clm
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/../cime_config/buildnml
  2023-06-01 22:11:13 ice 
Create namelist for component sice
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/CIME/non_py/src/components/stub_comps_nuopc/sice/cime_config/buildnml
  2023-06-01 22:11:13 ocn 
Create namelist for component socn
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/CIME/non_py/src/components/stub_comps_nuopc/socn/cime_config/buildnml
  2023-06-01 22:11:13 rof 
Create namelist for component mosart
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/../components/mosart//cime_config/buildnml
  2023-06-01 22:11:13 glc 
Create namelist for component sglc
   Calling /glade/u/home/guoqiang/CTSM_rep

submit_jobs case.run
Submit job case.run


Setting resource.RLIMIT_STACK to -1 from (-1, -1)
Generating namelists for /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100_iter0_trial0
  2023-06-01 22:11:15 atm 
Create namelist for component datm
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/../components/cdeps/datm/cime_config/buildnml
  2023-06-01 22:11:15 lnd 
Create namelist for component clm
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/../cime_config/buildnml
  2023-06-01 22:11:15 ice 
Create namelist for component sice
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/CIME/non_py/src/components/stub_comps_nuopc/sice/cime_config/buildnml
  2023-06-01 22:11:15 ocn 
Create namelist for component socn
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/CIME/non_py/src/components/stub_comps_nuopc/socn/cime_config/buildnml
  2023-06-01 22:11:15 rof 
Create namelist for component mosart
   Calling /glade/u/home/guoqiang/CTSM_repos/CTSM/cime/../components/mosart//cime_config

Exception from case_run: ERROR: RUN FAIL: Command 'mpirun `hostname`  -np 2 /glade/scratch/guoqiang/CTSM_outputs/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_0/CAMELS_0/bld/cesm.exe   >> cesm.log.$LID 2>&1 ' failed
See log file for details: /glade/scratch/guoqiang/CTSM_outputs/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100/CAMELS_100_iter0_trial0/run/cesm.log.230601-221114
Submit job case.st_archive
Cannot find a CAMELS_100_iter0_trial0.cpl*.r.*.nc file in directory /glade/scratch/guoqiang/CTSM_outputs/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100/CAMELS_100_iter0_trial0/run 


CompletedProcess(args='./case.submit --no-batch', returncode=0)

In [20]:
# move output to the archive folder
output_dir = xmlquery_output(path_CTSM_clone, 'DOUT_S_ROOT')
target_dir = f'{path_archive}/{caseflag}'
os.makedirs(str(pathlib.Path(target_dir).parent), exist_ok=True)
if os.path.isdir(target_dir):
    print(f'Warning!!! {target_dir} exists before moving archived files')
    _ = os.system(f'rm -r {target_dir}')

_ = subprocess.run(f'mv {output_dir} {target_dir}', shell=True)
_ = subprocess.run(f'cp {file_parameter_set} {target_dir}', shell=True)


In [29]:

# # delete cloned cases to save space
# if delete_clone:
#     os.chdir(path_CTSM_base)
#     # cimeoutroot = xmlquery_output(path_CTSM_clone, 'CIME_OUTPUT_ROOT')
#     # _ = subprocess.run(f'rm -r f{cimeoutroot}/{clone_case_name}')
#     RUNDIR = xmlquery_output(path_CTSM_clone, 'RUNDIR')
#     _ = subprocess.run(f'rm -r {str(pathlib.Path(RUNDIR).parent)}', shell=True)
#     _ = subprocess.run(f'rm -r {path_CTSM_clone}', shell=True)

In [32]:
import glob

In [33]:

# evaluate model results
infilelist = glob.glob(f'{path_archive}/{caseflag}/lnd/hist/*.clm2.h1.*.nc')
infilelist.sort()
fsurdat = get_parameter_from_Namelist_or_lndin('fsurdat', f'{path_CTSM_base}/user_nl_clm', f'{path_CTSM_base}/Buildconf/clmconf/lnd_in', type='str')
# fsurdat = '/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/shared_data_Sean/surfdata_CAMELS_split_nested_hist_78pfts_CMIP6_simyr2000_c230105.nc' # the information about AREA is needed from this file
date_start = '1994-10-01'
date_end = '1998-09-30'
ref_streamflow = '/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100_OstCalib/refdata/streamflow_data.csv'
add_flow_file = 'nofile'
kge_q, rmse_q = mo_evaluate(infilelist, fsurdat, date_start, date_end, ref_streamflow, add_flow_file)


Use streamflow reference file: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100_OstCalib/refdata/streamflow_data.csv
File does not exist: nofile
Remove it from add flow file list


In [38]:
# multi-objective evaluation of CTSM model outputs
# Guoqiang Tang
# Note: Here the average of outputs is used to compare to observed streamflow


import numpy as np
import xarray as xr
import pandas as pd
import sys, glob, os, subprocess

# turn off all warnings (not always necessary)
import warnings
warnings.filterwarnings("ignore")

########################################################################################################################
# define functions for calculating metrics

def ismember(a, b):
    bind = {}
    for i, elt in enumerate(b):
        if elt not in bind:
            bind[elt] = i
    ind = np.array([bind.get(itm, np.nan) for itm in a])
    ind1 = np.where(~np.isnan(ind))[0]
    ind2 = ind[ind1]
    return ind1.astype(int), ind2.astype(int) # None can be replaced by any other "not in b" value

def get_modified_KGE(obs,sim):
    sim[sim<0] = np.nan
    obs[obs<0] = np.nan
    ind = (~np.isnan(obs)) & (~np.isnan(sim))
    obs = obs[ind]
    sim = sim[ind]

    try:
        sd_sim=np.std(sim, ddof=1)
        sd_obs=np.std(obs, ddof=1)
        m_sim=np.mean(sim)
        m_obs=np.mean(obs)
        r=(np.corrcoef(sim,obs))[0,1]
        relvar=(float(sd_sim)/float(m_sim))/(float(sd_obs)/float(m_obs))
        bias=float(m_sim)/float(m_obs)
        kge=1.0-np.sqrt((r-1)**2 +(relvar-1)**2 + (bias-1)**2)
    except:
        kge = np.nan

    return kge


def get_RMSE(obs,sim):
    sim[sim<0] = np.nan
    obs[obs<0] = np.nan
    rmse = np.sqrt(np.nanmean(np.power((sim - obs),2)))
    return rmse

def get_mean_error(obs,sim):
    bias_err = np.nanmean(sim - obs)
    abs_err = np.nanmean(np.absolute(sim - obs))
    return bias_err, abs_err

########################################################################################################################
# define functions for reading CTSM outputs

def main_read_CTSM_streamflow(fsurdat, CTSMfilelist, date_start, date_end, clm_q_name, clm_q_sdim):
    ########################################################################################################################
    # read files
    ds_simu = xr.open_mfdataset(CTSMfilelist)
    ds_simu = ds_simu[[clm_q_name]]

    if date_start == 'default' or date_end == 'default':
        print(
            'Either date_start or date_end is default. Evaluation period will be the overlapped period of referene data and simulations')
    else:
        ds_simu = ds_simu.sel(time=slice(date_start, date_end))

    ds_simu = ds_simu.load()

    # change time format
    ds_simu['time'] = ds_simu.indexes['time'].to_datetimeindex()

    ########################################################################################################################
    # get the area of a basin to convert the unit of QRUNOFF from mm/s to m3/s

    with xr.open_dataset(fsurdat) as ds_surdat:
        area = ds_surdat.AREA.values

    # calculate streamflow: although mean is used, for Sean's setting, only one basin should be allowed effective in the calibration
    # streamflow? Use mean for this test
    ds_simu[clm_q_name].values = (ds_simu[clm_q_name].values / 1000) * (
                area * 1e6)  # raw q: mm/s; raw area km2; target: m3/s
    ds_simu = ds_simu.mean(dim=clm_q_sdim, skipna=True)

    return ds_simu


########################################################################################################################
# define functions for reading CAMELS data

def read_CAMELS_Q(file_Qobs):
    df_q_in = pd.read_csv(file_Qobs, delim_whitespace=True, header=None)
    years = df_q_in[1].values
    months = df_q_in[2].values
    days = df_q_in[3].values
    dates = [f'{years[i]}-{months[i]:02}-{days[i]:02}' for i in range(len(years))]
    q_obs = df_q_in[4].values * 0.028316847  # cfs to cms
    q_obs[q_obs < 0] = -9999.0
    df_q_out = pd.DataFrame({'Date': dates, 'Runoff_cms': q_obs})
    return df_q_out

def read_CAMELS_Q_and_to_xarray(ref_streamflow, ref_q_date, ref_q_name):
    ########################################################################################################################
    # load observation streamflow
    print('Use streamflow reference file:', ref_streamflow)
    df_q_obs = pd.read_csv(ref_streamflow)
    ds_q_obs = xr.Dataset()
    ds_q_obs.coords['time'] = pd.to_datetime(df_q_obs[ref_q_date].values)
    ds_q_obs[ref_q_name] = xr.DataArray(df_q_obs[ref_q_name].values, dims=['time']) # flexible time
    for i in range(10000):
        coli = ref_q_name + str(i)
        if coli in df_q_obs.columns:
            ds_q_obs[coli] = xr.DataArray(df_q_obs[coli].values, dims=['time'])  # flexible time
        else:
            break
    return ds_q_obs


def add_upstream_flow(add_flow_file, ds_simu, ref_q_date, ref_q_name, clm_q_name):
    ########################################################################################################################
    # add upstream flows to simulated streamflow

    add_flow_file = [f for f in add_flow_file.split(',') if len(f)>0]
    if len(add_flow_file) > 0:
        add_flow_file2 = []
        for f in add_flow_file:
            if not os.path.isfile(f):
                print('File does not exist:', f)
                print('Remove it from add flow file list')
            else:
                add_flow_file2.append(f)
        add_flow_file = add_flow_file2

    if len(add_flow_file) > 0:
        print('Flow files will be added to the incremental downstream basin:', add_flow_file)
        q_dd = np.zeros(len(ds_simu.time))
        num = np.zeros(len(ds_simu.time))
        time0 = ds_simu.time.values
        for i in range(len(add_flow_file)):
            df_addi = read_CAMELS_Q(add_flow_file[i])
            # df_addi = pd.read_csv(add_flow_file[i])
            timei = pd.to_datetime(df_addi[ref_q_date].values)
            ind1, ind2 = ismember(np.array(timei), time0)
            q_dd[ind2] = q_dd[ind2] + df_addi[ref_q_name].values[ind1]
            num[ind2] = num[ind2] + 1
        q_dd[num==0] = np.nan

        ds_simu[clm_q_name].values = ds_simu[clm_q_name].values + q_dd
        ratio = np.sum(~np.isnan(ds_simu[clm_q_name].values)) / len(ds_simu[clm_q_name].values)
        if ratio < 0.5:
            print('Warning!!!')
        print(f'The valid ratio of simulated streamflow is {ratio} after add upstream flow')

    return ds_simu

def mo_evaluate(outfile_metric, CTSMfilelist, fsurdat, date_start, date_end, ref_streamflow, add_flow_file=''):

    ######## default variable names
    clm_q_name = 'QRUNOFF' # default runoff variable name
    clm_q_sdim = 'lndgrid' # spatial dim name
    ref_q_name = 'Runoff_cms'
    ref_q_date = 'Date'

    ########################################################################################################################
    # load CTSM streamflow (m3/s)
    ds_simu = main_read_CTSM_streamflow(fsurdat, CTSMfilelist, date_start, date_end, clm_q_name, clm_q_sdim)

    ########################################################################################################################
    # load CAMELS observation streamflow (m3/s)
    ds_q_obs = read_CAMELS_Q_and_to_xarray(ref_streamflow, ref_q_date, ref_q_name)

    ########################################################################################################################
    # add upstream flows to simulated streamflow
    ds_simu = add_upstream_flow(add_flow_file, ds_simu, ref_q_date, ref_q_name, clm_q_name)

    ########################################################################################################################
    # evaluation

    ds_q_obs = ds_q_obs.sel(time=ds_q_obs.time.isin(ds_simu.time))
    ds_simu = ds_simu.sel(time=ds_simu.time.isin(ds_q_obs.time))

    kge_q = get_modified_KGE(obs=ds_q_obs[ref_q_name].values, sim=ds_simu[clm_q_name].values)
    rmse_q = get_RMSE(obs=ds_q_obs[ref_q_name].values, sim=ds_simu[clm_q_name].values)
    print(f'Evaluation result: kge_q={kge_q}, rmse_q={rmse_q}')

    ########################################################################################################################
    # write metric to file
    dfout = pd.DataFrame([[kge_q, rmse_q]], columns=['kge_q', 'rmse_q'])
    dfout.to_csv(outfile_metric, index=False)

    return kge_q, rmse_q



In [40]:

CTSMfilelist = glob.glob(f'/glade/scratch/guoqiang/moasmo_test/ctsm_outputs/iter0_trial2/lnd/hist/*.clm2.h1.*.nc')
CTSMfilelist.sort()
fsurdat = '/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/shared_data_Sean/surfdata_CAMELS_split_nested_hist_78pfts_CMIP6_simyr2000_c230105.nc'
date_start = '1994-10-01'
date_end = '1998-09-30'
ref_streamflow ='/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100_OstCalib/refdata/streamflow_data.csv'
add_flow_file='nofile'

In [45]:
    ######## default variable names
    clm_q_name = 'QRUNOFF' # default runoff variable name
    clm_q_sdim = 'lndgrid' # spatial dim name
    ref_q_name = 'Runoff_cms'
    ref_q_date = 'Date'

    ########################################################################################################################
    # load CTSM streamflow (m3/s)
    ds_simu = main_read_CTSM_streamflow(fsurdat, CTSMfilelist, date_start, date_end, clm_q_name, clm_q_sdim)

    ########################################################################################################################
    # load CAMELS observation streamflow (m3/s)
    ds_q_obs = read_CAMELS_Q_and_to_xarray(ref_streamflow, ref_q_date, ref_q_name)

    ########################################################################################################################
    # add upstream flows to simulated streamflow
    ds_simu = add_upstream_flow(add_flow_file, ds_simu, ref_q_date, ref_q_name, clm_q_name)

Use streamflow reference file: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100_OstCalib/refdata/streamflow_data.csv
File does not exist: nofile
Remove it from add flow file list


In [50]:
CTSMfilelist

['/glade/scratch/guoqiang/moasmo_test/ctsm_outputs/iter0_trial2/lnd/hist/CAMELS_100_iter0_trial2.clm2.h1.1993-10-01-00000.nc',
 '/glade/scratch/guoqiang/moasmo_test/ctsm_outputs/iter0_trial2/lnd/hist/CAMELS_100_iter0_trial2.clm2.h1.1994-10-01-00000.nc']

In [49]:
ds_simu

<xarray.Dataset>
Dimensions:  (time: 1)
Coordinates:
  * time     (time) datetime64[ns] 1994-10-01
Data variables:
    QRUNOFF  (time) float64 0.0

In [ ]:
    ########################################################################################################################
    # evaluation

    ds_q_obs = ds_q_obs.sel(time=ds_q_obs.time.isin(ds_simu.time))
    ds_simu = ds_simu.sel(time=ds_simu.time.isin(ds_q_obs.time))

    kge_q = get_modified_KGE(obs=ds_q_obs[ref_q_name].values, sim=ds_simu[clm_q_name].values)
    rmse_q = get_RMSE(obs=ds_q_obs[ref_q_name].values, sim=ds_simu[clm_q_name].values)
    print(f'Evaluation result: kge_q={kge_q}, rmse_q={rmse_q}')
